In [5]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

# 1. Read the CSV
df = pd.read_csv('nyc_open_space.csv')

# 2. Parse the 'the_geom' column from WKT to shapely geometries
df['geometry'] = df['the_geom'].apply(wkt.loads)

# 3. Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# 4. Optional: Select only useful columns
gdf = gdf[['PARK_NAME', 'LANDUSE', 'geometry']]

# 5. Project to a projected CRS (let's use EPSG:2263 — NYC State Plane feet)
gdf_proj = gdf.to_crs(epsg=2263)

# 6. Compute centroids in projected CRS
gdf_proj['centroid'] = gdf_proj.geometry.centroid

# 7. Reproject centroids back to WGS84 (lon/lat)
centroids_wgs84 = gdf_proj['centroid'].to_crs(epsg=4326)

# 8. Extract lon/lat from centroids
gdf['centroid_lon'] = centroids_wgs84.x
gdf['centroid_lat'] = centroids_wgs84.y

# 9. Save to GeoJSON
gdf.to_file('nyc_open_space.geojson', driver='GeoJSON')

print("GeoJSON saved as nyc_open_space.geojson")


GeoJSON saved as nyc_open_space.geojson


In [4]:
import geopandas as gpd

# 1. Load the ZIP GeoJSON
gdf = gpd.read_file('nyc_zip_codes.geojson')

# 2. Reproject to a projected CRS for accurate centroids (NY State Plane, EPSG:2263)
gdf_proj = gdf.to_crs(epsg=2263)

# 3. Calculate centroids
gdf_proj['centroid'] = gdf_proj.geometry.centroid

# 4. Reproject centroids back to WGS84 (EPSG:4326)
centroids_wgs84 = gdf_proj['centroid'].to_crs(epsg=4326)

# 5. Extract longitude and latitude
gdf['centroid_lon'] = centroids_wgs84.x
gdf['centroid_lat'] = centroids_wgs84.y

# 6. Save back to GeoJSON
gdf.to_file('nyc_zip_codes_with_centroids.geojson', driver='GeoJSON')

print("GeoJSON with centroids saved as nyc_zip_codes_with_centroids.geojson")


GeoJSON with centroids saved as nyc_zip_codes_with_centroids.geojson
